### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Check if there is null values and general information on the data set
purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Purchase ID  780 non-null    int64  
 1   SN           780 non-null    object 
 2   Age          780 non-null    int64  
 3   Gender       780 non-null    object 
 4   Item ID      780 non-null    int64  
 5   Item Name    780 non-null    object 
 6   Price        780 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


## Player Count

* Display the total number of players


In [3]:
#SN stands for Screen Name, and the unique value count of the column should give the number of players
SN_df=purchase_data["SN"].value_counts()


In [4]:
#Calculate the length of SN_df, which will give the total players count
total_players_count={"Total Players": [len(SN_df)]}
total_players_count_df=pd.DataFrame(total_players_count)
total_players_count_df


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
#Calculations over columns
number_of_purchases=len(purchase_data["Purchase ID"])
unique_game_counter=len(purchase_data["Item Name"].value_counts())
average_price= round(purchase_data["Price"].sum(axis=0)/len(purchase_data["Price"]),2)
total_revenue=purchase_data["Price"].sum(axis=0)

#Create a dictionary based on the calculations
summary_of_purchasing={"Number of Unique Items":[number_of_purchases],
                      "Average Price":[average_price],
                      "Number of Purchases": [number_of_purchases],
                      "Total Revenue": [total_revenue]}



In [6]:
#Convert the dictionary into a summary dataframe and map the correct $ value and formatting
purchasing_summary_df=pd.DataFrame(summary_of_purchasing)
purchasing_summary_df['Average Price'] = purchasing_summary_df['Average Price'].map('${:,.2f}'.format)
purchasing_summary_df['Total Revenue'] = purchasing_summary_df['Total Revenue'].map('${:,.2f}'.format)
purchasing_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,780,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [7]:
#Counting Gender Column Unique Values
#First extract the unique Players and then filter Gender with it
gamers_df=purchase_data[["SN","Gender"]]
unique_gamers_df=gamers_df.drop_duplicates(['SN','Gender'])

#Assign gender numbers to a new dataframe and format the Column names and Rows
gender_demographics_df=pd.DataFrame(unique_gamers_df["Gender"].value_counts())
gender_demographics_df=gender_demographics_df.rename(columns={"Gender":"Total Count"})


In [8]:
#Add a new percentage calculation column with the percent formatting
gender_demographics_df["Percentage of Players"]=gender_demographics_df.div(gender_demographics_df["Total Count"].sum(axis=0))
gender_demographics_df["Percentage of Players"]=gender_demographics_df["Percentage of Players"].map("{:.2%}".format)
gender_demographics_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
#Filter a new dataframe for Gender and Price and calculate prices grouping by Gender
purchase_data_filter_df=purchase_data[["Gender","Price"]]
count_df=purchase_data_filter_df.groupby("Gender", as_index=False).count()
avg_df=purchase_data_filter_df.groupby("Gender", as_index=False).mean()
sum_df=purchase_data_filter_df.groupby("Gender",as_index=False).sum()

#Merge Grouped Calculations into a new dataframe 
merge_1=pd.merge(count_df, avg_df, on="Gender")
merge_2=pd.merge(merge_1, sum_df, on="Gender")

#Rename Merged DataFrame Columns
purchasing_analysis_df=merge_2.rename(columns={"Price_x": "Purchase Count", "Price_y":"Average Purchase Price", "Price":"Total Purchase Value"})



In [10]:
#Copy Unique value dataframe to Calculate the Avg Total Purchase per Person Column
gender_demographics_df_copy=gender_demographics_df.copy()

#setting first name as index column and resetting index  
reset_df=gender_demographics_df_copy.reset_index()    
reset_df=reset_df.rename(columns={"index":"Gender"})
reset_df=reset_df[["Gender","Total Count"]]

#Merge the purchasing_analysis_df and reset_df
purchasing_analysis_df2=pd.merge(purchasing_analysis_df, reset_df, on="Gender")

#Calculate Avg Total Purchase per Person Column 
purchasing_analysis_df2["Avg Total Purchase per Person"]=purchasing_analysis_df2["Total Purchase Value"]/purchasing_analysis_df2["Total Count"]




In [11]:
#Format Final DataFrame for the Full Analysis
format_purchasing_analysis=purchasing_analysis_df2[["Gender","Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]
format_purchasing_analysis=format_purchasing_analysis.set_index('Gender')
format_purchasing_analysis["Average Purchase Price"]=format_purchasing_analysis["Average Purchase Price"].map('${:,.2f}'.format)
format_purchasing_analysis["Total Purchase Value"]=format_purchasing_analysis["Total Purchase Value"].map('${:,.2f}'.format)
format_purchasing_analysis["Avg Total Purchase per Person"]=format_purchasing_analysis["Avg Total Purchase per Person"].map('${:,.2f}'.format)

format_purchasing_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [12]:
#Filter gamers and their ages in a dataframe
unique_ages=purchase_data[["SN","Age"]]
unique_ages_df=unique_ages.drop_duplicates(["SN","Age"])

# Create the bins in which Data will be held 
bins = [0,9,14,19,24,29,34,39,100]

# this creates 8 bins as follows,
# (0,9], (9,14], (14,19], (19,24], (24,29], (29,34], (34,39], (39,100]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
unique_ages_df["Age Ranges"]=pd.cut(unique_ages_df["Age"], bins, labels=group_names)
unique_ages_df=unique_ages_df.sort_values(by=["Age"])
#Retrieve the Count data and calculate Percentage of Players
age_count_df=unique_ages_df.groupby("Age Ranges", as_index=False).count()
age_count_df=age_count_df[["Age Ranges","SN" ]]

#Format the DataFrame
age_count_df=age_count_df.rename(columns={"SN": "Total Count"})
age_count_df["Percentage of Players"]=age_count_df["Total Count"]/age_count_df["Total Count"].sum()
age_count_df["Percentage of Players"]= age_count_df["Percentage of Players"].map("{:.2%}".format)
df=age_count_df.set_index('Age Ranges')
df.rename_axis(" ")


<ipython-input-12-425ffa690be6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_ages_df["Age Ranges"]=pd.cut(unique_ages_df["Age"], bins, labels=group_names)


,Total Count,Percentage of Players
,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [13]:
#Filter gamers and their ages in a dataframe
bins = [0,9,14,19,24,29,34,39,100]

# Create the names for the bins

group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Ranges"]=pd.cut(purchase_data["Age"], bins, labels=group_names)
purchase_data_byage=purchase_data.copy()
purchase_data_byage=purchase_data_byage.sort_values(by=["Age"])

#Calculate Price Avgs,Totals, Counts
count_purchase_byage= purchase_data_byage.groupby("Age Ranges", as_index=False).count()
avg_purchase_byage= purchase_data_byage.groupby("Age Ranges", as_index=False).mean()
sum_purchase_byage= purchase_data_byage.groupby("Age Ranges", as_index=False).sum()

merge_11=pd.merge(count_purchase_byage, avg_purchase_byage, on="Age Ranges")
merge_22=pd.merge(merge_11, sum_purchase_byage, on="Age Ranges")
merge_22=merge_22[["Age Ranges","SN" ,"Price_y", "Price"]]
merge_22=merge_22.rename(columns={"SN":"Purchase Count", "Price_y": "Average Purchase Price", "Price":"Total Purchase Value"})

#Arranging and Formatting the DataFrame
purch_analysis_age_df=pd.merge(merge_22, age_count_df, on="Age Ranges")
purch_analysis_age_df["Avg Total Purchase per Person"]=purch_analysis_age_df["Total Purchase Value"]/purch_analysis_age_df["Total Count"]
purch_analysis_age_df=purch_analysis_age_df[["Age Ranges", "Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]
purch_analysis_age_df["Total Purchase Value"]=purch_analysis_age_df["Total Purchase Value"].map('${:,.2f}'.format)
purch_analysis_age_df["Average Purchase Price"]=purch_analysis_age_df["Average Purchase Price"].map('${:,.2f}'.format)
purch_analysis_age_df["Avg Total Purchase per Person"]=purch_analysis_age_df["Avg Total Purchase per Person"].map('${:,.2f}'.format)

purch_analysis_age_df.set_index("Age Ranges")

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [14]:
#Create a new dataframe from the main dataframe and filter for the top spenders

top_spender=purchase_data[["SN","Price"]]

count_top_spender=top_spender.groupby("SN", as_index=False).count()
avg_top_spender=top_spender.groupby("SN", as_index=False).mean()
sum_top_spender=top_spender.groupby("SN", as_index=False).sum()


merge_111=pd.merge(count_top_spender, avg_top_spender, on="SN")
merge_222=pd.merge(merge_111, sum_top_spender, on="SN")

merge_222=merge_222.rename(columns={"Price_x":"Purchase Count", "Price_y": "Average Purchase Price", "Price":"Total Purchase Value"})
merge_222=merge_222.sort_values(by=["Total Purchase Value"],ascending=False)
merge_222["Average Purchase Price"]=merge_222["Average Purchase Price"].map('${:,.2f}'.format)
merge_222["Total Purchase Value"]=merge_222["Total Purchase Value"].map('${:,.2f}'.format)
top_spenders_formatted_df=merge_222.set_index("SN")
top_spenders_formatted_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
# Filter for the Item information 
most_popular_df=purchase_data[["Item ID","Item Name","Price"]]

cnt_most_popular_gb=most_popular_df.groupby(["Item ID", "Item Name"]).count()
avg_most_popular_gb=most_popular_df.groupby(["Item ID", "Item Name"]).mean()
sum_most_popular_gb=most_popular_df.groupby(["Item ID", "Item Name"]).sum()

cnt_most_popular_gb=cnt_most_popular_gb.rename(columns={"Price":"Purchase Count"})
avg_most_popular_gb=avg_most_popular_gb.rename(columns={"Price":"Item Price"})
sum_most_popular_gb=sum_most_popular_gb.rename(columns={"Price":"Total Purchase Value"})

#Concatanate the Calculated Columns keeping the Multi Index Structure and Format them

df_concat1=pd.concat([cnt_most_popular_gb, avg_most_popular_gb], axis=1)
df_concat2=pd.concat([df_concat1, sum_most_popular_gb], axis=1)
most_popular_game=df_concat2.sort_values(by=["Purchase Count"],ascending=False)
most_popular_game["Item Price"]=most_popular_game["Item Price"].map('${:,.2f}'.format)
most_popular_game["Total Purchase Value"]=most_popular_game["Total Purchase Value"].map('${:,.2f}'.format)
most_popular_game.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [16]:
# Use the pre sorted most popular game dataframe (df_concat2) and sort for the most profitable game

most_profit_df=df_concat2.sort_values(by=["Total Purchase Value"],ascending=False)
most_profit_df["Item Price"]=most_profit_df["Item Price"].map('${:,.2f}'.format)
most_profit_df["Total Purchase Value"]=most_profit_df["Total Purchase Value"].map('${:,.2f}'.format)
most_profit_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
